In [ ]:
# -*- coding: utf-8 -*-
"""
Project: Personal Finance Tracker
Description: A comprehensive desktop application to track, manage, and visualize personal finances.
This application fulfills the project requirements by demonstrating object-oriented design,
file I/O, robust exception handling, and integration with an external API for currency conversion.
"""

import tkinter as tk
from tkinter import ttk, messagebox, simpledialog
import csv
import os
from datetime import datetime
import requests
import pandas as pd
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import matplotlib.pyplot as plt

# --- Constants ---
TRANSACTIONS_CSV = 'financial_transactions.csv'
API_KEY_FILE = 'exchangerate_api_key.txt'
SUPPORTED_CURRENCIES = ['USD', 'EUR', 'GBP', 'JPY', 'CAD', 'AUD', 'CHF', 'CNY', 'INR', 'BRL', 'RUB']
INCOME_CATEGORIES = ['Salary', 'Freelance', 'Investment', 'Gift', 'Business', 'Other']
EXPENSE_CATEGORIES = ['Food & Groceries', 'Housing & Rent', 'Utilities', 'Transportation', 'Healthcare', 'Entertainment', 'Shopping', 'Education', 'Other']

# --- Data Layer Class ---
class TransactionManager:
    """Handles all file I/O operations for financial transactions."""
    def __init__(self, filename=TRANSACTIONS_CSV):
        """Initializes the TransactionManager and ensures the CSV file exists."""
        self.filename = filename
        self._ensure_file_exists()

    def _ensure_file_exists(self):
        """Creates the transaction file with headers if it's not present."""
        if not os.path.exists(self.filename):
            try:
                with open(self.filename, 'w', newline='', encoding='utf-8') as f:
                    writer = csv.writer(f)
                    writer.writerow(['Date', 'Description', 'Type', 'Category', 'Amount', 'Currency'])
            except IOError as e:
                messagebox.showerror("Fatal File Error", f"Could not create the transaction file:\n{e}\nPlease check your folder permissions.")
                raise

    def add_transaction(self, transaction_data):
        """Appends a new transaction record to the CSV file."""
        try:
            with open(self.filename, 'a', newline='', encoding='utf-8') as f:
                writer = csv.writer(f)
                writer.writerow(transaction_data)
        except IOError as e:
            messagebox.showerror("File Write Error", f"Failed to save transaction: {e}")
            raise

    def load_all_transactions(self):
        """Reads all transactions from the CSV file, skipping the header."""
        if not os.path.exists(self.filename):
            return []
        try:
            with open(self.filename, 'r', newline='', encoding='utf-8') as f:
                reader = csv.reader(f)
                next(reader) # Skip header
                return list(reader)
        except (IOError, csv.Error) as e:
            messagebox.showerror("File Read Error", f"Failed to load transactions: {e}")
            return []

# --- API Layer Class ---
class CurrencyConverterAPI:
    """Manages API interactions for currency conversion."""
    def __init__(self, api_key):
        self.api_key = api_key
        self.base_url = f"https://v6.exchangerate-api.com/v6/{self.api_key}/latest/"

    def get_exchange_rates(self, base_currency):
        """Fetches a dictionary of exchange rates for a given base currency."""
        if not self.api_key:
            return None, "API Key is missing."
        try:
            response = requests.get(self.base_url + base_currency, timeout=10)
            response.raise_for_status()
            data = response.json()
            if data.get("result") == "success":
                return data["conversion_rates"], None
            else:
                error_type = data.get("error-type", "Unknown API Error")
                return None, f"API Error: {error_type}"
        except requests.exceptions.Timeout:
            return None, "The request to the currency server timed out."
        except requests.exceptions.RequestException as e:
            return None, f"Network Error: Could not connect.\n{e}"

# --- Main Application Class ---
class FinanceTrackerApp:
    """The main application class, orchestrating the GUI and business logic."""

    def __init__(self, root):
        self.root = root
        self.root.title("Personal Finance Tracker")
        self.root.geometry("1300x800")
        self.root.minsize(1100, 650)
        self.root.protocol("WM_DELETE_WINDOW", self._on_closing)

        self._configure_styles()

        self.transaction_manager = TransactionManager()
        self.api_key = self._load_api_key()
        self.currency_converter = CurrencyConverterAPI(self.api_key)

        self._create_widgets()
        self.refresh_all_data()

        if not self.api_key:
            self.root.after(100, self._prompt_for_api_key)

    def _configure_styles(self):
        """Configures custom styles for ttk widgets for a modern look."""
        self.style = ttk.Style()
        self.style.theme_use('clam')
        self.style.configure("TNotebook", background="#f0f0f0", borderwidth=0)
        self.style.configure("TNotebook.Tab", font=('Helvetica', 12, 'bold'), padding=[10, 5], background="#d0d0d0")
        self.style.map("TNotebook.Tab", background=[("selected", "#ffffff")], foreground=[("selected", "#000000")])
        self.style.configure("Treeview.Heading", font=('Helvetica', 11, 'bold'))
        self.style.configure("Treeview", rowheight=25, font=('Helvetica', 10))
        self.style.configure("Success.TButton", foreground="white", background="#28a745", font=('Helvetica', 10, 'bold'))
        self.style.configure("Primary.TButton", foreground="white", background="#007bff", font=('Helvetica', 10, 'bold'))

    def _load_api_key(self):
        """Loads API key from a local file, with exception handling."""
        try:
            if os.path.exists(API_KEY_FILE):
                with open(API_KEY_FILE, 'r') as f:
                    return f.read().strip()
        except IOError as e:
            messagebox.showwarning("API Key Warning", f"Could not read API key file: {e}")
        return None

    def _save_api_key(self, key):
        """Saves the user's API key to a local file, with exception handling."""
        try:
            with open(API_KEY_FILE, 'w') as f:
                f.write(key)
            self.api_key = key
            self.currency_converter.api_key = key
            messagebox.showinfo("API Key Saved", "API Key has been saved successfully.")
            self.refresh_all_data() # Refresh summary after getting key
        except IOError as e:
            messagebox.showerror("API Key Error", f"Failed to save API key: {e}")

    def _prompt_for_api_key(self):
        """Displays a dialog to ask the user for their API key."""
        key = simpledialog.askstring("API Key Required",
                                     "Welcome! To use currency features, please enter your free API key from ExchangeRate-API.com:",
                                     parent=self.root)
        if key:
            self._save_api_key(key)
        else:
            messagebox.showwarning("Functionality Limited", "The currency converter and summary will not function without a valid API key.")

    def _create_widgets(self):
        """Creates and arranges all GUI elements in the main window."""
        main_notebook = ttk.Notebook(self.root)
        main_notebook.pack(pady=15, padx=15, expand=True, fill='both')

        transactions_tab = ttk.Frame(main_notebook, padding=10)
        main_notebook.add(transactions_tab, text='Dashboard')
        self._create_dashboard_tab(transactions_tab)

        viz_tab = ttk.Frame(main_notebook, padding=10)
        main_notebook.add(viz_tab, text='Spending Analysis')
        self._create_visualization_tab(viz_tab)

        converter_tab = ttk.Frame(main_notebook, padding=10)
        main_notebook.add(converter_tab, text='Currency Converter')
        self._create_converter_tab(converter_tab)

    def _create_dashboard_tab(self, parent):
        """Creates the content for the main 'Dashboard' tab."""
        input_frame = ttk.LabelFrame(parent, text="Add New Transaction", padding=15)
        input_frame.pack(side='left', fill='y', padx=(0, 10), anchor='n')

        display_frame = ttk.Frame(parent)
        display_frame.pack(side='right', fill='both', expand=True)

        ttk.Label(input_frame, text="Date:").grid(row=0, column=0, padx=5, pady=5, sticky='w')
        self.date_entry = ttk.Entry(input_frame)
        self.date_entry.insert(0, datetime.now().strftime('%Y-%m-%d'))
        self.date_entry.grid(row=0, column=1, padx=5, pady=5, sticky='ew')

        ttk.Label(input_frame, text="Description:").grid(row=1, column=0, padx=5, pady=5, sticky='w')
        self.desc_entry = ttk.Entry(input_frame)
        self.desc_entry.grid(row=1, column=1, padx=5, pady=5, sticky='ew')

        ttk.Label(input_frame, text="Type:").grid(row=2, column=0, padx=5, pady=5, sticky='w')
        self.type_var = tk.StringVar(value='Expense')
        self.type_combo = ttk.Combobox(input_frame, textvariable=self.type_var, values=['Expense', 'Income'], state='readonly')
        self.type_combo.grid(row=2, column=1, padx=5, pady=5, sticky='ew')
        self.type_combo.bind('<<ComboboxSelected>>', self._update_categories_dropdown)

        ttk.Label(input_frame, text="Category:").grid(row=3, column=0, padx=5, pady=5, sticky='w')
        self.category_var = tk.StringVar()
        self.category_combo = ttk.Combobox(input_frame, textvariable=self.category_var, state='readonly')
        self.category_combo.grid(row=3, column=1, padx=5, pady=5, sticky='ew')
        self._update_categories_dropdown()

        ttk.Label(input_frame, text="Amount:").grid(row=4, column=0, padx=5, pady=5, sticky='w')
        self.amount_entry = ttk.Entry(input_frame)
        self.amount_entry.grid(row=4, column=1, padx=5, pady=5, sticky='ew')

        ttk.Label(input_frame, text="Currency:").grid(row=5, column=0, padx=5, pady=5, sticky='w')
        self.currency_var = tk.StringVar(value='USD')
        self.currency_combo = ttk.Combobox(input_frame, textvariable=self.currency_var, values=SUPPORTED_CURRENCIES, state='readonly')
        self.currency_combo.grid(row=5, column=1, padx=5, pady=5, sticky='ew')

        add_btn = ttk.Button(input_frame, text="Add Transaction", command=self.add_transaction, style="Success.TButton")
        add_btn.grid(row=6, column=0, columnspan=2, pady=20, sticky='ew')

        tree_frame = ttk.Frame(display_frame)
        tree_frame.pack(fill='both', expand=True)

        columns = ('Date', 'Description', 'Type', 'Category', 'Amount', 'Currency')
        self.tree = ttk.Treeview(tree_frame, columns=columns, show='headings')
        for col in columns:
            self.tree.heading(col, text=col)
            self.tree.column(col, width=120, anchor='center')
        self.tree.column('Description', width=200, anchor='w')

        v_scroll = ttk.Scrollbar(tree_frame, orient='vertical', command=self.tree.yview)
        h_scroll = ttk.Scrollbar(tree_frame, orient='horizontal', command=self.tree.xview)
        self.tree.configure(yscrollcommand=v_scroll.set, xscrollcommand=h_scroll.set)

        v_scroll.pack(side='right', fill='y')
        h_scroll.pack(side='bottom', fill='x')
        self.tree.pack(side='left', fill='both', expand=True)

        summary_frame = ttk.LabelFrame(display_frame, text="Financial Summary (USD Equivalent)", padding=10)
        summary_frame.pack(fill='x', pady=(10, 0))
        self.summary_label = ttk.Label(summary_frame, text="Calculating...", font=('Helvetica', 12, 'bold'))
        self.summary_label.pack()

    def _create_visualization_tab(self, parent):
        """Creates the content for the 'Spending Analysis' tab."""
        self.fig = Figure(figsize=(8, 6), dpi=100, facecolor='#f0f0f0')
        self.ax = self.fig.add_subplot(111)
        self.canvas = FigureCanvasTkAgg(self.fig, master=parent)
        self.canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=True)
        self.ax.set_title("Expense Breakdown by Category", fontsize=14, weight='bold')

    def _create_converter_tab(self, parent):
        """Creates the content for the 'Currency Converter' tab."""
        converter_frame = ttk.LabelFrame(parent, text="Real-Time Currency Conversion", padding=20)
        converter_frame.pack(padx=20, pady=20)

        ttk.Label(converter_frame, text="Amount:").grid(row=0, column=0, padx=5, pady=10)
        self.conv_amount_entry = ttk.Entry(converter_frame, width=15, font=('Helvetica', 12))
        self.conv_amount_entry.grid(row=0, column=1, padx=5, pady=10)

        ttk.Label(converter_frame, text="From:").grid(row=0, column=2, padx=5, pady=10)
        self.from_currency_var = tk.StringVar(value='USD')
        self.from_currency_combo = ttk.Combobox(converter_frame, textvariable=self.from_currency_var, values=SUPPORTED_CURRENCIES, width=10, state='readonly')
        self.from_currency_combo.grid(row=0, column=3, padx=5, pady=10)

        ttk.Label(converter_frame, text="To:").grid(row=0, column=4, padx=5, pady=10)
        self.to_currency_var = tk.StringVar(value='EUR')
        self.to_currency_combo = ttk.Combobox(converter_frame, textvariable=self.to_currency_var, values=SUPPORTED_CURRENCIES, width=10, state='readonly')
        self.to_currency_combo.grid(row=0, column=5, padx=5, pady=10)

        convert_btn = ttk.Button(converter_frame, text="Convert", command=self.perform_conversion, style="Primary.TButton")
        convert_btn.grid(row=1, column=0, columnspan=6, pady=20, sticky='ew')

        self.conversion_result_label = ttk.Label(converter_frame, text="", font=('Helvetica', 16, 'bold'), foreground="#007bff")
        self.conversion_result_label.grid(row=2, column=0, columnspan=6, pady=20)

    def _update_categories_dropdown(self, event=None):
        """Updates the category dropdown based on the selected transaction type."""
        current_values = self.category_combo['values']
        new_values = INCOME_CATEGORIES if self.type_var.get() == 'Income' else EXPENSE_CATEGORIES
        if current_values != new_values:
            self.category_combo['values'] = new_values
            self.category_combo.set(new_values[0])

    def add_transaction(self):
        """Validates input, adds transaction via manager, and refreshes UI."""
        try:
            date_str = self.date_entry.get()
            datetime.strptime(date_str, '%Y-%m-%d')
            desc = self.desc_entry.get()
            category = self.category_var.get()
            amount = float(self.amount_entry.get())

            if not desc or not category:
                raise ValueError("Description and Category cannot be empty.")
            if amount <= 0:
                raise ValueError("Amount must be a positive number.")
        except ValueError as e:
            messagebox.showerror("Invalid Input", f"Please check your input values.\nError: {e}")
            return

        try:
            transaction_data = [date_str, desc, self.type_var.get(), category, amount, self.currency_var.get()]
            self.transaction_manager.add_transaction(transaction_data)
        except Exception:
            return

        messagebox.showinfo("Success", "Transaction added successfully.")
        self._clear_input_form()
        self.refresh_all_data()

    def _clear_input_form(self):
        """Resets the transaction input form to default values."""
        self.date_entry.delete(0, tk.END)
        self.date_entry.insert(0, datetime.now().strftime('%Y-%m-%d'))
        self.desc_entry.delete(0, tk.END)
        self.amount_entry.delete(0, tk.END)
        self.type_combo.set('Expense')
        self._update_categories_dropdown()
        self.currency_combo.set('USD')

    def refresh_all_data(self):
        """Central method to reload and update all data displays."""
        transactions = self.transaction_manager.load_all_transactions()
        self._update_treeview(transactions)
        self._update_summary(transactions)
        self._update_plot(transactions)

    def _update_treeview(self, transactions):
        """Populates the transaction Treeview with fresh data."""
        self.tree.delete(*self.tree.get_children())
        for row in transactions:
            self.tree.insert('', tk.END, values=row)

    def _update_summary(self, transactions):
        """Calculates and displays financial summary, converting all amounts to USD."""
        if not self.api_key:
            self.summary_label.config(text="API Key needed for multi-currency summary.")
            return

        self.summary_label.config(text="Fetching rates for summary...")
        rates, error = self.currency_converter.get_exchange_rates('USD')
        if error:
            self.summary_label.config(text=f"Summary unavailable: {error}")
            return

        total_income, total_expense = 0.0, 0.0
        for trans in transactions:
            try:
                amount, currency, trans_type = float(trans[4]), trans[5], trans[2]
                amount_usd = amount / rates[currency] if currency != 'USD' else amount
                if trans_type == 'Income':
                    total_income += amount_usd
                else:
                    total_expense += amount_usd
            except (ValueError, IndexError, KeyError):
                continue

        balance = total_income - total_expense
        summary_text = f"Total Income: ${total_income:,.2f} | Total Expenses: ${total_expense:,.2f} | Balance: ${balance:,.2f}"
        self.summary_label.config(text=summary_text)

    def _update_plot(self, transactions):
        """Updates the pie chart visualization of expenses by category."""
        self.ax.clear()
        if not transactions:
            self.ax.text(0.5, 0.5, 'No data to generate a chart.', ha='center', va='center')
        else:
            try:
                df = pd.DataFrame(transactions, columns=['Date', 'Description', 'Type', 'Category', 'Amount', 'Currency'])
                df['Amount'] = pd.to_numeric(df['Amount'], errors='coerce')
                expense_df = df[(df['Type'] == 'Expense') & (df['Amount'] > 0)]

                if expense_df.empty:
                    self.ax.text(0.5, 0.5, 'No expense data to display.', ha='center', va='center')
                else:
                    category_spending = expense_df.groupby('Category')['Amount'].sum()
                    self.ax.pie(category_spending, labels=category_spending.index, autopct='%1.1f%%', startangle=140, pctdistance=0.85)
                    centre_circle = plt.Circle((0, 0), 0.70, fc='#f0f0f0')
                    self.ax.add_artist(centre_circle)
            except Exception as e:
                self.ax.text(0.5, 0.5, f'Error plotting data:\n{e}', ha='center', va='center')

        self.ax.set_title("Expense Breakdown by Category", fontsize=14, weight='bold')
        self.canvas.draw()

    def perform_conversion(self):
        """Handles the currency conversion logic in the converter tab."""
        try:
            amount = float(self.conv_amount_entry.get())
            from_curr = self.from_currency_var.get()
            to_curr = self.to_currency_var.get()
        except ValueError:
            messagebox.showerror("Invalid Input", "Please enter a valid number for the amount.")
            return

        rates, error = self.currency_converter.get_exchange_rates(from_curr)
        if error:
            messagebox.showerror("API Error", error)
            self.conversion_result_label.config(text="Conversion failed")
            return

        if to_curr in rates:
            converted_amount = amount * rates[to_curr]
            result_text = f"{amount:,.2f} {from_curr} = {converted_amount:,.2f} {to_curr}"
            self.conversion_result_label.config(text=result_text)
        else:
            messagebox.showerror("Conversion Error", f"The target currency '{to_curr}' is not available.")

    def _on_closing(self):
        """Handles the application closing event."""
        if messagebox.askokcancel("Quit", "Do you want to exit the Personal Finance Tracker?"):
            self.root.destroy()

# --- Main execution block ---
if __name__ == "__main__":
    root = tk.Tk()
    app = FinanceTrackerApp(root)
    root.mainloop()